In [ ]:
Geo Spatial analysis 2

In [7]:
import pandas as pd

# Load cleaned dataset
df = pd.read_csv("US_Accidents_Cleaned.csv")

C:\Users\polep\AppData\Local\Temp\ipykernel_14652\2814253055.py:4: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("US_Accidents_Cleaned.csv")


In [5]:
import pandas as pd
import plotly.express as px
import streamlit as st

In [9]:
def plot_accidents_sampled(df, location_col='State', sample_size=300000):
    counts = df[location_col].value_counts().head(5)
    top5_locations = counts.index.tolist()
    df['is_top5'] = df[location_col].isin(top5_locations)

    # Sample data to reduce size
    if len(df) > sample_size:
        df_sample = df.sample(sample_size, random_state=42)
    else:
        df_sample = df

    fig = px.scatter(
        data_frame=df_sample,
        x='Start_Lng',
        y='Start_Lat',
        color='is_top5',
        labels={'Start_Lng': 'Longitude', 'Start_Lat': 'Latitude', 'is_top5': 'Top 5 Accident-Prone'},
        title=f"Accident Scatter Plot Highlighting Top 5 {location_col}s",
        opacity=0.6,
        hover_data=[location_col]
    )
    st.plotly_chart(fig)


In [ ]:
Hypothesis Testing

In [ ]:
1.What time of day has the most accidents?

In [ ]:
# Ensure Start_Time is datetime
df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')

# Extract hour from datetime
df['Hour'] = df['Start_Time'].dt.hour
hourly_counts = df['Hour'].value_counts().sort_index()
import plotly.express as px

fig = px.bar(
    x=hourly_counts.index,
    y=hourly_counts.values,
    labels={'x': 'Hour of Day', 'y': 'Number of Accidents'},
    title='Accidents by Hour of Day',
    text=hourly_counts.values
)
fig.update_layout(xaxis=dict(tickmode='linear'))
st.plotly_chart(fig)

In [ ]:
2. Are accidents more severe during rain or fog?

In [ ]:
# Standardize weather column
df['Weather_Condition'] = df['Weather_Condition'].str.lower()

# Filter only rain or fog conditions
weather_accidents = df[df['Weather_Condition'].str.contains('rain|fog', na=False)]
severity_by_weather = weather_accidents.groupby('Weather_Condition')['Severity'].mean().sort_values(ascending=False)
print(severity_by_weather)
import plotly.express as px

fig = px.bar(
    severity_by_weather,
    x=severity_by_weather.index,
    y=severity_by_weather.values,
    labels={'x': 'Weather Condition', 'y': 'Average Severity'},
    title='Accident Severity during Rain vs Fog'
)
fig.show()

In [ ]:
3. Is there a correlation between visibility and severity?

In [ ]:
# Drop rows with missing or zero visibility
df_clean = df.dropna(subset=['Visibility(mi)', 'Severity'])
df_clean = df_clean[df_clean['Visibility(mi)'] > 0]
correlation = df_clean['Visibility(mi)'].corr(df_clean['Severity'])
print(f"Correlation between visibility and severity: {correlation:.3f}")
import plotly.express as px

fig = px.scatter(
    df_clean,
    x='Visibility(mi)',
    y='Severity',
    trendline='ols',  # shows linear trend
    labels={'Visibility(mi)': 'Visibility (miles)', 'Severity': 'Accident Severity'},
    title='Severity vs Visibility'
)
fig.show()